In [13]:
import numpy as np
from math import log10, floor

# When we are doing forward elimination, we round all the values to a certain significant digit that we are testing,
# although it is not perfect, it still shows that as our significant digit goes down, the non pivot method experiences more
# rounding errors compared to the pivot methods

# Show that partial pivoting is better because it also helps prevent divide by 0 errors caused by our computers

# First in class, I will show the method explained in the text book
# Explain that Python floating point numbers are double precision, that they are 16 or 17 significant digits
# It is hard to find a big enough matrix that we can use to show error for when it is so precise
# Therefore, I've used round_sig function to round our calculations to a certain sig digit and use that as a way to stimulate such a system
# For a simple problem like this non pivot already has rounding errors, if it were a bigger problem, there would be even bigger rounding errors
# This proves that partial pivoting, swapping rows so that the bigger one in the column can go to the diagonal, can help prevent rounding errors
# caused by adding and subtracting very small and very big numbers.

# Maybe show one step by step example of how both of our Gaussian Elimination works
# Show significant digits 3 through 7 and show a table with errors

def round_sig(x, sig=2):
    # because python supports up to 16 significant digits, it is hard to find a small
    # problem that may produce a rounding error, so instaed, i round each calculation
    # to a certain significant digit to see these rounding errors

    # obviously, not perfect representation of floating numbers with certain sig digit but it proves our experiment
    try:
        # taken from stackoverflow, tested it and it works as expected
        return round(x, sig-int(floor(log10(abs(x))))-1)
    except:
        return 0

def forward_elimination_no_pivot(A, b, n, s):
    # A, b make the augmented matrix
    # n is the number of Rows
    # s is what significant digit we are rounding to
    for row in range(0, n-1):
        for i in range(row+1, n):
            factor = A[i,row] / A[row,row]
            for j in range(row, n):
                A[i,j] = round_sig(A[i,j], s) - round_sig(factor * A[row,j], s)
            b[i] = round_sig(b[i], s) - round_sig(factor * b[row], s)
        # print('Foward Eliminated Row: \nA = \n%s and b = %s' % (A,b))
    return A, b

def forward_elimination_pivot(A, b, n, s):
    # A, b make the augmented matrix
    # n is the number of Rows
    # s is what significant digit we are rounding to
    for row in range(0, n-1):
        max_index = row
        pivot_max = A[row][row]
        for i in range(row+1, n):
            if abs(A[row][i]) > pivot_max:
                pivot_max = A[row][i]
                max_index = i
        if max_index != row:
            swap_row(A, b, row, max_index)
        for i in range(row+1, n):
            factor = A[i,row] / A[row,row]
            for j in range(row, n):
                A[i,j] = round_sig(A[i,j], s) - round_sig(factor * A[row,j], s)
            b[i] = round_sig(b[i], s) - round_sig(factor * b[row], s)
        # print('Foward Eliminated Row: \nA = \n%s and b = %s' % (A,b))
    return A, b

def back_substitution(a, b, n):
    x = np.zeros((n,1))
    x[n-1] = b[n-1] / a[n-1, n-1]
    for row in range(n-2, -1, -1):
        sums = b[row]
        for j in range(row+1, n):
            sums = sums - a[row,j] * x[j]
        x[row] = sums / a[row,row]
    return x

def swap_row(A, b, index1, index2):
    # print("\nSwapping Rows at index " + str(index1) + " and " + str(index2) + ": ")
    for i in range(len(A[0])):
        temp = A[index2][i]
        A[index2][i] = A[index1][i]
        A[index1][i] = temp
    # print('A = \n' + str(A))
    temp = b[index2]
    b[index2] = b[index1]
    b[index1] = temp
    # print('b = \n' + str(b))

def gauss_no_pivot(A, b, s):
    # This function performs Gauss elimination without pivoting.
    print("\n\n################ Gaussian Elimination with no Pivot: #################")
    if 0 in A.diagonal():
        print("Division by 0, can't be done without pivoting\n")
        return

    n = A.shape[0]
    A, b = forward_elimination_no_pivot(A, b, n, s)
    return back_substitution(A, b, n)

def gauss_pivot(A, b, s):
    # This function performs Gauss elimination with partial pivoting.
    n = A.shape[0]
    print("\n\n################## Gaussian Elimination with Pivot: ##################")
    A, b = forward_elimination_pivot(A, b, n, s)
    return back_substitution(A, b, n)

if __name__ == '__main__':
    # A = np.array([  [0, 1, 1],
    #                 [1, 0, 1],
    #                 [1, 1, 0]])
    # b = np.array([1, 1, 1])
    A = np.array([  [4.44891, 0.11131],
                    [0.00111, 0.00018]])
                    
    b = np.array([0.111129, 0.999911])
    print("Input A: \n" + str(A))
    print("Input b: \n" + str(b))

    # Gaussian Elimination with both methods at 7 significant digits
    A1 = np.copy(A)
    A2 = np.copy(A)
    b1 = np.copy(b)
    b2 = np.copy(b)

    x = gauss_pivot(A1, b1, 7)
    print('Gaussian with pivot at 7 significant digit result is x = \n %s' % x)

    x = gauss_no_pivot(A2, b2, 7)
    print('Gaussian with no pivot at 7 significant digit result is x = \n %s' % x)

    # Gaussian Elimination with both methods at 6 significant digits
    A1 = np.copy(A)
    A2 = np.copy(A)
    b1 = np.copy(b)
    b2 = np.copy(b)

    x = gauss_pivot(A1, b1, 6)
    print('Gaussian with pivot at 6 significant digit result is x = \n %s' % x)

    x = gauss_no_pivot(A2, b2, 6)
    print('Gaussian with no pivot at 6 significant digit result is x = \n %s' % x)

    # Gaussian Elimination with both methods at 5 significant digits
    A1 = np.copy(A)
    A2 = np.copy(A)
    b1 = np.copy(b)
    b2 = np.copy(b)

    x = gauss_pivot(A1, b1, 5)
    print('Gaussian with pivot at 5 significant digit result is x = \n %s' % x)

    x = gauss_no_pivot(A2, b2, 5)
    print('Gaussian with no pivot at 5 significant digit result is x = \n %s' % x)

    # Gaussian Elimination with both methods at 4 significant digits
    A1 = np.copy(A)
    A2 = np.copy(A)
    b1 = np.copy(b)
    b2 = np.copy(b)

    x = gauss_pivot(A1, b1, 4)
    print('Gaussian with pivot at 4 significant digit result is x = \n %s' % x)

    x = gauss_no_pivot(A2, b2, 4)
    print('Gaussian with no pivot at 4 significant digit result is x = \n %s' % x)

    # Gaussian Elimination with both methods at 3 significant digits
    A1 = np.copy(A)
    A2 = np.copy(A)
    b1 = np.copy(b)
    b2 = np.copy(b)

    x = gauss_pivot(A1, b1, 3)
    print('Gaussian with pivot at 3 significant digit result is x = \n %s' % x)

    x = gauss_no_pivot(A2, b2, 3)
    print('Gaussian with no pivot at 3 significant digit result is x = \n %s' % x)

Input A: 
[[4.44891e+00 1.11310e-01]
 [1.11000e-03 1.80000e-04]]
Input b: 
[0.111129 0.999911]


################## Gaussian Elimination with Pivot: ##################
Gaussian with pivot at 7 significant digit result is x = 
 [[-164.31176131]
 [6568.31701577]]


################ Gaussian Elimination with no Pivot: #################
Gaussian with no pivot at 7 significant digit result is x = 
 [[-164.31176131]
 [6568.31701577]]


################## Gaussian Elimination with Pivot: ##################
Gaussian with pivot at 6 significant digit result is x = 
 [[-164.3117937 ]
 [6568.31831027]]


################ Gaussian Elimination with no Pivot: #################
Gaussian with no pivot at 6 significant digit result is x = 
 [[-164.3117937 ]
 [6568.31831027]]


################## Gaussian Elimination with Pivot: ##################
Gaussian with pivot at 5 significant digit result is x = 
 [[-164.31184519]
 [6568.32036813]]


################ Gaussian Elimination with no Pivot: #########